# ML-3. Обучение с учителем: классификация

# 1. Введение

В этом модуле мы продолжаем знакомство с моделями МО в области обучения с учителем. На этот раз поговорим о задаче классификации. Вспомним, где находится классификация на нашей карте машинного обучения:

![](https://lms.skillfactory.ru/assets/courseware/v1/2c3b5a00129b4dbedff6ed9fd2995cdb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml1-3_2.png)

Вначале мы снова обратимся к классу линейных моделей и рассмотрим **логистическую регрессию**.

Затем поговорим о **деревьях решений** для задачи классификации и научимся строить из этих деревьев целый лес.

### ЦЕЛИ ДАННОГО МОДУЛЯ:

* Познакомиться с принципами работы модели логистической регрессии для решения задачи классификации. 
* Рассмотреть метрики классификации и научиться оценивать качество моделей, решающих данную задачу.
* Узнать принципы построения деревьев решений.
* Изучить основы ансамблевых моделей типа бэггинг на примере случайного леса.
* Научиться применять деревья решений и случайные леса для решения задачи классификации

# 2. Логистическая регрессия

✍ Ранее мы обсуждали модель линейной регрессии, которая предназначена для решения задачи регрессии. Теперь нам предстоит разобраться с тем, как преобразовать данную модель, чтобы она решала задачу классификации.

Для начала вспомним, что такое классификация.

> **Задача классификации (classification)** — задача, в которой мы пытаемся предсказать класс объекта на основе признаков в наборе данных. То есть задача сводится к предсказанию целевого признака, который является **категориальным**.

![](https://lms.skillfactory.ru/assets/courseware/v1/f1c52bc11a0e81a47e3096dfb2750fdc/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml1-3_9.png)

Когда классов, которые мы хотим предсказать, только два, классификация называется бинарной. Например, мы можем предсказать, болен ли пациент раком, является ли изображение человеческим лицом, является ли письмо спамом и т. д.

Когда классов, которые мы хотим предсказать, более двух, классификация называется **мультиклассовой (многоклассовой)**. Например, предсказание модели самолёта по радиолокационным снимкам, классификация животных на фотографиях, определение языка, на котором говорит пользователь, разделение писем на группы.

→ Для простоты мы пока разберёмся с бинарной классификацией, а в следующем юните обобщим результат на мультиклассовую.

Что вообще означает «решить задачу классификации»? Это значит построить разделяющую поверхность в пространстве признаков, которая делит пространство на части, каждая из которых соответствует определённому классу. 

Ниже представлены примеры разделяющих поверхностей, которые производят бинарную классификацию. Красным и синим цветом обозначены классы, зелёным — собственно поверхность, которая делит пространство признаков на две части. В каждой из этих частей находятся только наблюдения определённого класса.

![](https://lms.skillfactory.ru/assets/courseware/v1/295dc860fdc114b9449701a16f7b6200/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_1.png)

Ю Модели, которые решают задачу классификации, называются классификаторами (classifier).

Если взять в качестве разделяющей поверхности некоторую плоскость (ровная поверхность на первом рисунке), то мы получаем модель логистической регрессии, которая тесно связана с рассмотренной нами ранее линейной регрессией.

Давайте для начала вспомним, как выглядит уравнение модели линейной регрессии в общем случае:

$$\hat{y}=w_{0}+w_{1} x_{1}+w_{2} x_{2}+\ldots+w_{m} x_{m}=w_{0}+\sum_{j=1}^{m} w_{j} x_{j}$$

В общем случае это уравнение гиперплоскости, которая стремится приблизить зависимость целевой переменной от  факторов.

* Когда фактор всего один, уравнение задаёт прямую:

$$\hat{y} = w_{0} + w_{1}x$$

* Когда факторов два, уравнение задаёт плоскость:

$$\hat{y} = w_{0} + w_{1}x_{1} + w_{2}x_{2}$$

→ Но всё это работает только в том случае, когда целевой признак $y$, который мы хотим предсказать, является числовым, например цена, вес, время аренды и т. д.

Что же делать с этой моделью, когда целевой признак  является категориальным? Например, является письмо спамом или обычным письмом?

Можно предположить, что, раз у нас есть две категории, мы можем обозначить категории за $y=1$ (Спам) и $y=0$ (Не спам) и обучить линейную регрессию предсказывать 0 и 1.

Но результат будет очень плохим. Выглядеть это будет примерно так:

![](https://lms.skillfactory.ru/assets/courseware/v1/a41a936873f275372356f53254e213d5/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_2.png)



Для больших значений  прямая будет выдавать значения больше 1, а для очень маленьких — меньше 0. Что это значит? Непонятно. Непонятно и то, что делать со значениями в диапазоне от 0 до 1. Да, можно относить значения на прямой выше 0.5 к классу 1, а меньше либо равным 0.5 — к классу 0, но это всё «костыли».

Идея! Давайте переведём задачу классификации в задачу регрессии. Вместо предсказания класса будем предсказывать вероятность принадлежности к этому классу. 

Модель должна выдавать некоторую вероятность $P$, которая будет определять, принадлежит ли данный объект к классу 1: например, вероятность того, что письмо является спамом. При этом вероятность того, что письмо является обычным письмом (класс 0), определяется как $Q=1-P$.  

Когда модель будет обучена на предсказание вероятности, мы зададим некоторый порог вероятности. Если предсказанная вероятность будет выше этого порога, мы определим объект к классу 1, а если ниже — к классу 0.

Например, стандартный порог равен 0.5. То есть если вероятность $P>0.5$, мы будем считать письмо спамом, а если $P \leq 0.5$ — обычным информативным письмом.

В итоге мы добьёмся того, что будем предсказывать не дискретный категориальный, а непрерывный числовой признак, который лежит в диапазоне [0, 1]. А это уже знакомая нам задача регрессии.

→ Однако остался главный вопрос: как научить модель предсказывать вероятности, ведь они должны лежать строго в диапазоне от 0 до 1, а предсказания линейной регрессии лежат в диапазоне от $-\infty$ до $+\infty$? 

Тут-то мы и приходим к модели логистической регрессии — **регрессии вероятностей**.

## ОБЩЕЕ ПРЕДСТАВЛЕНИЕ О ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

> **Логистическая регрессия (Logistic Regression)** — одна из простейших моделей для решения задачи классификации. Несмотря на простоту, модель входит в топ часто используемых алгоритмов классификации в Data Science.

В основе логистической регрессии лежит логистическая функция (logistic function) $\sigma(z)$ — отсюда и название модели. Однако более распространённое название этой функции — **сигмόида (sigmoid)**. Записывается она следующим образом:

$$\sigma(z) = \frac{1}{1+e^{-z}}$$

А вот график её зависимости от аргумента $z$:

![](https://lms.skillfactory.ru/assets/courseware/v1/17e5df54243ab349c94672502f9f4dd0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_3.png)

**В чём преимущество этой функции?**

У сигмоиды есть два очень важных для нас свойства:

* Значения сигмоиды $\sigma(z)$ лежат в диапазоне от 0 до 1 при любых значения аргумента $z$: какой бы $z$ вы ни подставили, число меньше 0 или больше 1 вы не получите.
* Сигмоида выдаёт значения $\sigma(z)>0.5$ при её аргументе $z>0$, $\sigma(z)<0.5$ — при $z<0$ и $\sigma(z)=0.5$ — при $z=0$.
 

Это ведь и есть свойства вероятности! Выходом сигмоиды является число от 0 до 1, которое можно интерпретировать как вероятность принадлежности к классу 1. Её мы и пытаемся предсказать.

**Основная идея** модели логистической регрессии: возьмём модель линейной регрессии (обозначим её выход за $z$)

$$z=w_{0}+\sum_{j=1}^{m} w_{j} x_{j}$$

и подставим выход модели $z$ в функцию сигмоиды, чтобы получить искомые оценки вероятности (в математике принято писать оценочные величины с «шапкой» наверху, а истинные значения — без «шапки», это чистая формальность):

$$\hat{P}=\sigma(z)=\frac{1}{1+e^{-z}}=\frac{1}{1+e^{-w_{0}-\sum_{j=1}^{m} w_{j} x_{j}}}=\frac{1}{1+e^{-\bar{w} \cdot \bar{x}}}$$

**Примечание**. Далее в модуле мы будем называть оценки вероятности $\hat{P}$ просто вероятностью, но только для краткости. Это не значит, что эти оценки являются истинными вероятностями принадлежности к каждому из классов (их нельзя сосчитать, так как для этого нужна выборка бесконечного объёма). Если вы употребляете термин «вероятности» на собеседованиях, обязательно предварительно укажите, что вы подразумеваете оценку вероятности.

Обучать будем всё в совокупности, пытаясь получить наилучшую оценку вероятности $\hat{P}$. Если вероятность $\hat{P}>0.5$, относим объект к классу 1, а если $\hat{P} \leq 0.5$, относим объект к классу 0. 

Математически это записывается следующей формулой:

$$\hat{y}=I[\hat{P}]=\left\{\begin{array}{l} 1, \hat{P}>0.5 \\ 0, \hat{P} \leq 0.5 \end{array}\right.$$

**Примечание**. В данном выражении $I[ \hat{P}]$ называется индикаторной функцией. Она возвращает 1, если её значение больше какого-то порога, и 0 — в противном случае. Математики часто записывают просто квадратные скобки, опуская символ $I$: $[ \hat{P}]$.

**Чего мы добились таким преобразованием?**

Если мы обучим модель, то есть подберём  коэффициенты $w_{0}, w_{1}, w_{2}, ..., w_{m}$ (как их найти, обсудим чуть позже) таким образом, что для объектов класса 1 модель линейной регрессии начнёт выдавать положительное число, а для класса 0 — выдавать отрицательное число, то тогда, подставив предсказание линейной регрессии $z$ в сигмоиду, мы сможем получать вероятности принадлежности к каждому из классов в диапазоне от 0 до 1.

Далее по порогу вероятности мы сможем определять, к какому классу принадлежит объект.

Это и есть наша цель. Мы свели задачу классификации к задаче регрессии для предсказания вероятностей. 

Для бинарной классификации описанное выше будет выглядеть следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/8e663f443f23b393252ae85246b5d23f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_4.png)



Рассмотрим, как это работает, на примере.

Пусть мы каким-то образом обучили модель линейной регрессии предсказывать положительные числа для спам-писем и отрицательные — для обычных писем.

Подаём характеристики письма $x_{1}, x_{2}, ..., x_{m}$ в выражение для линейной регрессии и получаем ответ модели, например $z=1.5$. Тогда, подставив его в сигмоиду, получим:

$$\hat{P}=\sigma(z)=\frac{1}{1+e^{-z}}=\frac{1}{1+e^{-1.5}}=0.82$$

Таким образом, вероятность того, что данный объект принадлежит классу спама, равна 0.82, что больше порогового значения 0.5. То есть мы относим данное письмо к спаму: $\hat{y}=1$.

Пусть теперь мы подали на вход модели характеристики другого письма и получили $z=-0.91$. Тогда, подставив этот результат в сигмоиду, получим:

$$\hat{P}=\sigma(z)=\frac{1}{1+e^{-z}}=\frac{1}{1+e^{-0.91}}=0.28$$

Вероятность того, что данный объект принадлежит классу спама, равна 0.28, что меньше порогового значения 0.5. Мы относим данное письмо к обычным письмам: $\hat{y}=0$.

Кстати, вероятность того, что это письмо будет обычным, равна противоположной вероятности: $Q=1 - 0.28 = 0.72$. 

Полученное выражение для оценки вероятности $\hat{P}$ и будет называться моделью логистической регрессии:

$$\hat{P}=\frac{1}{1+e^{-w_{0}-\sum_{j=1}^{m} w_{j} x_{j}}}$$

$$\hat{y}=I[\hat{P}]$$



**Разберёмся с геометрией**

Возьмём частный случай, когда класс объекта зависит от двух признаков — $x_1$ и $x_2$.

Рассмотрим пример.

Мы пытаемся предсказать поступление студента в университет в зависимости от результатов двух экзаменов. Целевой признак $y$ — результат поступления в аспирантуру (admission outcome) с двумя возможными значениями: поступил или не поступил. Факторы: $x_1$ — результат сдачи первого экзамена (Exam1 Score) и $x_2$ — результат сдачи второго (Exam 2 Score). Будем предсказывать вероятность поступления с помощью логистической регрессии.

Изобразим зависимость в пространстве двух факторов (вид сверху) в виде диаграммы рассеяния, а целевой признак отобразим в виде точек (непоступившие) и крестиков (поступившие).

Если рассматривать уравнение линейной регрессии отдельно от сигмоиды, то геометрически построить логистическую регрессию на основе двух факторов — значит найти такие коэффициенты $w_0$, $w_1$ и $w_2$ уравнения плоскости, при которых наблюдается наилучшее разделение пространства на две части.

$$z = w_{0} + w_{1}x_{1} + w_{2}x_{2}$$

Тогда выражение для $z$ будет задавать в таком пространстве плоскость (в проекции вида сверху — прямую), которая разделяет всё пространство на две части. Над прямой вероятность поступления будет $>0.5$, а под прямой $<0.5$:

![](https://lms.skillfactory.ru/assets/courseware/v1/a8209513f0cfc3b554f494d79165269c/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_5.png)

Коэффициенты построенной выше плоскости равны (как их найти, обсудим позже):

$$w_{0} = -25.05$$

$$w_{1} = 0.205$$

$$w_{2} = 0.2$$

Тогда модель логистической регрессии будет иметь вид:

$$z = -25.05+0.205x_{1} + 0.2x_{2}$$

$$\hat{P} = \sigma(z) = \frac{1}{1+e^{-z}}$$

Появляется новый абитуриент, и мы хотим предсказать вероятность его поступления. Баллы студента: $x_{1}=67$, $x_{2}=53$. Заметьте, что точка с такими координатами находится ниже нашей плоскости (то есть абитуриент, скорее всего, не поступит).

Тогда:

$$z = -25.05+0.205 \cdot 67 + 0.2 \cdot 53 = -0.71$$

$$\hat{P} = \sigma(z) = \frac{1}{1+e^{-0.71}} = 0.32$$

Итак, оценка вероятности поступления студента составляет 0.32, то есть его шанс поступления составляет 32 %.

А что если мы возьмём точку, лежащую выше прямой?

Например, появился абитуриент с баллами $x_{1}=80$, $x_{2}=75$. Подставим его баллы в нашу модель логистической регрессии, чтобы понять, какова оценочная вероятность поступления:

$$z = -25.05+0.205 \cdot 80 + 0.2 \cdot 75 = 6.34$$

$$\hat{P} = \sigma(z) = \frac{1}{1+e^{-6.34}} = 0.99$$

Таким образом, оценка вероятности поступления абитуриента составляет 0.99, шанс поступления — 99 %.

**В чём математический секрет?**

Математически подстановка в уравнение плоскости точки, которая не принадлежит ей (находится ниже или выше), означает вычисление расстояния от этой точки до плоскости.

* Если точка находится ниже плоскости, расстояние будет отрицательным ($z<0$).
* Если точка находится выше плоскости, расстояние будет положительным ($z>0$).
* Если точка находится на самой плоскости, $z=0$.  

Мы знаем, что подстановка отрицательных чисел в сигмоиду приведёт к вероятности , а постановка положительных — к вероятности . 

Таким образом, ключевым моментом в предсказании логистической регрессии является расстояние от точки до разделяющей плоскости в пространстве факторов. Это расстояние в литературе часто называется **отступом (margin)**. 

В этом и состоит секрет работы логистической регрессии.

Чем больше расстояние от точки, находящейся выше разделяющей плоскости, до самой плоскости, тем больше оценка вероятности принадлежности к классу 1.

Попробуйте подставить различные координаты точек в модель логистической регрессии и убедитесь в этом.

Можно построить тепловую карту, которая показывает, чему равны вероятности в каждой точке пространства:

![](https://lms.skillfactory.ru/assets/courseware/v1/30baeed1eeb22b64aa28d75952115e87/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_7.png)

На рисунке точки, которые относятся к классу непоступивших абитуриентов, лежащие ниже разделяющей плоскости, находятся в красной зоне. Чем насыщеннее красный цвет, тем ниже вероятность того, что абитуриент поступит в аспирантуру. И наоборот, точки, которые относятся к классу поступивших абитуриентов, лежащие выше разделяющей плоскости, находятся в синей зоне. Чем насыщеннее синий цвет, тем выше вероятность того, что абитуриент поступит в аспирантуру.

Для случая зависимости целевого признака $y$ от трёх факторов $x_1$, $x_2$ и $x_3$, например от баллов за два экзамена и рейтинга университета, из которого выпустился абитуриент, выражение для  будет иметь вид:

$$z = w_{0} + w_{1}x_{1} + w_{2}x_{2} + w_{3}x_{3}$$

Уравнение задаёт плоскость в четырёхмерном пространстве. Но если вспомнить, что $y$ — категориальный признак и классы можно обозначить цветом, то получится перейти в трёхмерное пространство. Разделяющая плоскость будет выглядеть следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/ecda661b626060bcc2c4dc6593249013/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml3-2_8.png)

В общем случае, когда у нас есть зависимость от $m$ факторов, линейное выражение, находящееся под сигмоидой, будет обозначать **разделяющую гиперплоскость**.

$$z = w_{0} + w_{1}x_{1} + w_{2}x_{2} + ...+ w_{m}x_{m} = w_{0} + \sum_{j=1}^{m} w_{j}x_{j}$$

### Задание 2.5

Решается задача бинарной классификации с помощью логистической регрессии.

Задано уравнение разделяющей плоскости:

$$z = 1+5x_{1}-0.3x_{2}$$

На вход алгоритма подаётся новое наблюдение с признаками $x_1=0.3$ и $x_2=3$. Определите вероятность принадлежности наблюдения к классу 1. Ответ округлите до двух знаков после точки-разделителя.

In [6]:
import math

x_1 = 0.3
x_2 = 3
e = math.e

z = 1 + 5*x_1 - 0.3*x_2

p = 1/(1 + e**(-z))
round(p, 2)

0.83

### ПОИСК ПАРАМЕТРОВ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

Итак, мы разобрались с тем, как выглядит модель логистической регрессии и что она означает в геометрическом смысле.

Но остался один главный вопрос: как найти такие коэффициенты $w=(w_{0}, w_{1}, w_{2}, ..., w_{m})$, чтобы гиперплоскость разделяла пространство наилучшим образом?

Вновь обратимся к нашей схеме минимизации эмпирического риска:

![](https://lms.skillfactory.ru/assets/courseware/v1/5fac5fe11d423f674949523e3db643c9/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-ml1-2_1.png)

Можно предположить, что стоит использовать метод наименьших квадратов. Введём функцию ошибки — средний квадрат разности MSE между истинными классами $y$ и предсказанными классами $\hat{y}$ и попытаемся его минимизировать.

Сразу можно достоверно предсказать, что результат такого решения будет плохим, поэтому воздержимся от его использования.

Здесь нужен другой подход. Это метод **максимального правдоподобия (Maximum Likelihood Estimation — MLE)**. 

> **Правдоподобие** — это оценка того, насколько вероятно получить истинное значение целевой переменной $y$ при данных $x$ и параметрах $w$. 

Данный метод позволяет получить функцию правдоподобия.

**Цель метода** — найти такие параметры $w=(w_{0}, w_{1}, w_{2}, ..., w_{m})$, в которых наблюдается максимум функции правдоподобия. Подробнее о выводе формулы вы можете прочитать [здесь](https://habr.com/ru/post/485872/).

А мы пока что опустим математические детали метода и приведём только конечную формулу:

$$likelihood = \sum_{i}^{n} (y_{i} log (\hat{P_{i}}) + (1-y_{i}) log (1-\hat{P_{i}})) \rightarrow max_{w}$$

Не пугайтесь. Давайте разберёмся, что есть что и как работает эта функция.

* $n$ — количество наблюдений.
* $y_i$ — это истинный класс (1 или 0) для -ого объекта из набора данных.
* $\hat{P_{i}} = \sigma(z_{i})$ — предсказанная с помощью логистической регрессии вероятность принадлежности к классу 1 для $i$-ого объекта из набора данных.
* $z_i$ — результат подстановки $i$-ого объекта из набора данных в уравнение разделяющей плоскости $z_{i}= \bar{w} \cdot \bar{x_{i}}$.
* $log$ — логарифм (обычно используется натуральный логарифм по основанию $e - ln$).

Пример расчёта функции правдоподобия

Вернёмся к примеру с абитуриентами. Пусть у нас есть выборка из четырёх студентов с оценками по двум экзаменам:  и . Возьмём уравнение разделяющей плоскости, которое мы использовали ранее:

Мы взяли всех студентов из выборки в формулу сигмоиды и получили оценочную вероятность поступления каждого из студентов:

i                                   |1	    |2	    |3	    |4
--------                            |-------|-------|-------|------
$\hat{P_i}$ — оценка вероятности	|0.2	|0.8	|1	    |0.6
$y_i$ — истинный класс	            |0	    |0	    |1	    |1
 
Подсчитаем, чему равна функция правдоподобия при данных предсказаниях вероятностей:

$$\begin{gathered} \text { likelihood }=\sum_{i}^{n}\left(y_{i} \log \left(\hat{P_{i}}\right)+\left(1-y_{i}\right) \log (1-\hat{P})\right)= \\ =((0 \log (0.2)+(1-0) \log (1-0.2))+(0 \log (0.8)+(1-0) \log (1-0.8))+ \\ +(1 \log (1)+(1-1) \log (1-1))+(1 \log (0.6)+(1-1) \log (1-0.6)))= \\ =(\log (0.8)+\log (0.2)+\log (1)+\log (0.6))=2.34 \end{gathered}$$
